
# Predicting the Dow Jones with News

# Imports

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import median_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
import matplotlib.pyplot as plt
import os

C:\softwares\conda4.8.2-python3.7\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\softwares\conda4.8.2-python3.7\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
C:\softwares\conda4.8.2-python3.7\lib\site-packages\numpy\.libs\libopenblas.noijjg62emaszi6nyurl6jbkm4evbgm7.gfortran-win_amd64.dll
C:\softwares\conda4.8.2-python3.7\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\softwares\conda4.8.2-python3.7\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)
C:\softwares\conda4.8.2-python3.7\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\softwares\conda4.

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\samra\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# Keras Imports
from keras.models import Sequential
from keras import initializers
from keras.layers import Dropout, Activation, Embedding, Convolution1D, MaxPooling1D, Input, Dense, add, \
                         BatchNormalization, Flatten, Reshape, Concatenate
from keras.layers.recurrent import LSTM, GRU
from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, SGD, RMSprop
from keras import regularizers
from keras.utils.vis_utils import plot_model
import re

Using TensorFlow backend.


In [4]:

import os
from pathlib import Path
import os.path as osp

In [5]:
data_path = osp.join(Path(os.getcwd()).parent,'Data')
dj = pd.read_csv(data_path  + "/stock_original.csv")
news = pd.read_csv(data_path  + "/news_modified.csv")

## Inspect the data

In [6]:
dj.head()

,Date,Open,High,Low,Close,Volume,Adj Close,cap
0,2008-08-08,11432.089844,11759.959961,11388.040039,11734.320312,212830000.0,11734.320312,297.580078
1,2008-08-11,11729.669922,11867.110352,11675.530273,11782.349609,183190000.0,11782.349609,52.030273
2,2008-08-12,11781.700195,11782.349609,11601.519531,11642.469727,173590000.0,11642.469727,-148.890625
3,2008-08-13,11632.809570,11633.780273,11453.339844,11532.959961,182550000.0,11532.959961,-100.739258
4,2008-08-14,11532.070312,11718.280273,11450.889648,11615.929688,159790000.0,11615.929688,79.139649


In [7]:
dj.isnull().sum() #No missing data

Date         0
Open         0
High         0
Low          0
Close        0
Volume       0
Adj Close    0
cap          0
dtype: int64

In [8]:
news.isnull().sum() #No missing data

Date    0
News    0
dtype: int64

In [9]:
news.head()

,Date,News
0,08-06-2008,"b'Marriage, they said, was reduced to the stat..."
1,08-06-2008,b'Nim Chimpsky: The tragedy of the chimp who t...
2,08-06-2008,"b""Canada: Beware slippery slope' to censorship..."
3,08-06-2008,b'EU Vice-President Luisa Morgantini and the I...
4,08-06-2008,"b""Israeli minister: Israel will attack Iran if..."


In [10]:
print(dj.shape)
print(news.shape)

(3036, 8)
(106071, 2)


In [11]:
# Compare the number of unique dates. We want matching values.
print(len(set(dj.Date)))
print(len(set(news.Date)))

3036
3847


In [62]:
# Remove the extra dates that are in news
news = news[news.Date.isin(dj.Date)]

In [64]:
news

,Date,News


In [13]:
print(len(set(dj.Date)))
print(len(set(news.Date)))

3036
0


In [14]:
# Remove unwanted features - keep the 'Open' price only
dj = dj.drop(['High','Low','Close','Volume','Adj Close'], 1)
dj.head()

,Date,Open,cap
0,2008-08-08,11432.089844,297.580078
1,2008-08-11,11729.669922,52.030273
2,2008-08-12,11781.700195,-148.890625
3,2008-08-13,11632.809570,-100.739258
4,2008-08-14,11532.070312,79.139649


In [15]:
# Calculate the difference in opening prices between the following and current day.
# The model will try to predict the change in Open value based on the today's news.
dj = dj.set_index('Date')
dj.head()

,Open,cap
Date,,
2008-08-08,11432.089844,297.580078
2008-08-11,11729.669922,52.030273
2008-08-12,11781.700195,-148.890625
2008-08-13,11632.809570,-100.739258
2008-08-14,11532.070312,79.139649


In [16]:
# Target variable = Tomorrow's Open Price - Today's Open Price
dj = -1 * dj.diff(periods=1)

In [17]:
dj.head()

,Open,cap
Date,,
2008-08-08,NaN,NaN
2008-08-11,-297.580078,245.549805
2008-08-12,-52.030273,200.920898
2008-08-13,148.890625,-48.151367
2008-08-14,100.739258,-179.878907


In [18]:
dj['Date'] = dj.index
dj = dj.reset_index(drop=True)

In [19]:
dj.head()

,Open,cap,Date
0,NaN,NaN,2008-08-08
1,-297.580078,245.549805,2008-08-11
2,-52.030273,200.920898,2008-08-12
3,148.890625,-48.151367,2008-08-13
4,100.739258,-179.878907,2008-08-14


In [20]:
# Remove top row since it has a null value.
dj = dj[dj.Open.notnull()]

In [21]:
# Check if there are any more null values.
dj.isnull().sum()

Open    0
cap     0
Date    0
dtype: int64

In [65]:
dj['Date'] = pd.to_datetime(dj['Date']).dt.strftime('%d-%m-%Y')
news['Date'] = pd.to_datetime(news['Date'])

## Combine the two datasets - For each date, get all the headlines and the price

In [61]:
# Create a list of the opening prices and their corresponding daily headlines from the news
# Define/Initialize the variables
price = []
headlines = []

# For all the rows in the dataframe
for row in dj.iterrows():
    # define a new variable to store all the headlines for the day
    daily_headlines = []
    # Spot the date in the given row
    date = row[1]['Date']
    
    print('date ',date)
    # Store the price for the date
    price.append(row[1]['Open'])
    
    
    for row_ in news[news.Date==date].iterrows():
        print(' date ',row_[1]['News'])
        daily_headlines.append(row_[1]['News'])
    
    print('daily_headlines ',daily_headlines)
    # Append the headlines for the date
    headlines.append(daily_headlines)
    # Track progress
    if len(price) % 500 == 0:
        print(len(price))

date  2008-08-11
daily_headlines  []
date  2008-08-12
daily_headlines  []
date  2008-08-13
daily_headlines  []
date  2008-08-14
daily_headlines  []
date  2008-08-15
daily_headlines  []
date  2008-08-18
daily_headlines  []
date  2008-08-19
daily_headlines  []
date  2008-08-20
daily_headlines  []
date  2008-08-21
daily_headlines  []
date  2008-08-22
daily_headlines  []
date  2008-08-25
daily_headlines  []
date  2008-08-26
daily_headlines  []
date  2008-08-27
daily_headlines  []
date  2008-08-28
daily_headlines  []
date  2008-08-29
daily_headlines  []
date  2008-09-02
daily_headlines  []
date  2008-09-03
daily_headlines  []
date  2008-09-04
daily_headlines  []
date  2008-09-05
daily_headlines  []
date  2008-09-08
daily_headlines  []
date  2008-09-09
daily_headlines  []
date  2008-09-10
daily_headlines  []
date  2008-09-11
daily_headlines  []
date  2008-09-12
daily_headlines  []
date  2008-09-15
daily_headlines  []
date  2008-09-16
daily_headlines  []
date  2008-09-17
daily_headlines  []
d

daily_headlines  []
date  2010-06-10
daily_headlines  []
date  2010-06-11
daily_headlines  []
date  2010-06-14
daily_headlines  []
date  2010-06-15
daily_headlines  []
date  2010-06-16
daily_headlines  []
date  2010-06-17
daily_headlines  []
date  2010-06-18
daily_headlines  []
date  2010-06-21
daily_headlines  []
date  2010-06-22
daily_headlines  []
date  2010-06-23
daily_headlines  []
date  2010-06-24
daily_headlines  []
date  2010-06-25
daily_headlines  []
date  2010-06-28
daily_headlines  []
date  2010-06-29
daily_headlines  []
date  2010-06-30
daily_headlines  []
date  2010-07-01
daily_headlines  []
date  2010-07-02
daily_headlines  []
date  2010-07-06
daily_headlines  []
date  2010-07-07
daily_headlines  []
date  2010-07-08
daily_headlines  []
date  2010-07-09
daily_headlines  []
date  2010-07-12
daily_headlines  []
date  2010-07-13
daily_headlines  []
date  2010-07-14
daily_headlines  []
date  2010-07-15
daily_headlines  []
date  2010-07-16
daily_headlines  []
date  2010-07-19
d

daily_headlines  []
date  2011-04-29
daily_headlines  []
date  2011-05-02
daily_headlines  []
date  2011-05-03
daily_headlines  []
date  2011-05-04
daily_headlines  []
date  2011-05-05
daily_headlines  []
date  2011-05-06
daily_headlines  []
date  2011-05-09
daily_headlines  []
date  2011-05-10
daily_headlines  []
date  2011-05-11
daily_headlines  []
date  2011-05-12
daily_headlines  []
date  2011-05-13
daily_headlines  []
date  2011-05-16
daily_headlines  []
date  2011-05-17
daily_headlines  []
date  2011-05-18
daily_headlines  []
date  2011-05-19
daily_headlines  []
date  2011-05-20
daily_headlines  []
date  2011-05-23
daily_headlines  []
date  2011-05-24
daily_headlines  []
date  2011-05-25
daily_headlines  []
date  2011-05-26
daily_headlines  []
date  2011-05-27
daily_headlines  []
date  2011-05-31
daily_headlines  []
date  2011-06-01
daily_headlines  []
date  2011-06-02
daily_headlines  []
date  2011-06-03
daily_headlines  []
date  2011-06-06
daily_headlines  []
date  2011-06-07
d

daily_headlines  []
date  2012-05-14
daily_headlines  []
date  2012-05-15
daily_headlines  []
date  2012-05-16
daily_headlines  []
date  2012-05-17
daily_headlines  []
date  2012-05-18
daily_headlines  []
date  2012-05-21
daily_headlines  []
date  2012-05-22
daily_headlines  []
date  2012-05-23
daily_headlines  []
date  2012-05-24
daily_headlines  []
date  2012-05-25
daily_headlines  []
date  2012-05-29
daily_headlines  []
date  2012-05-30
daily_headlines  []
date  2012-05-31
daily_headlines  []
date  2012-06-01
daily_headlines  []
date  2012-06-04
daily_headlines  []
date  2012-06-05
daily_headlines  []
date  2012-06-06
daily_headlines  []
date  2012-06-07
daily_headlines  []
date  2012-06-08
daily_headlines  []
date  2012-06-11
daily_headlines  []
date  2012-06-12
daily_headlines  []
date  2012-06-13
daily_headlines  []
date  2012-06-14
daily_headlines  []
date  2012-06-15
daily_headlines  []
date  2012-06-18
daily_headlines  []
date  2012-06-19
daily_headlines  []
date  2012-06-20
d

daily_headlines  []
date  2013-04-22
daily_headlines  []
date  2013-04-23
daily_headlines  []
date  2013-04-24
daily_headlines  []
date  2013-04-25
daily_headlines  []
date  2013-04-26
daily_headlines  []
date  2013-04-29
daily_headlines  []
date  2013-04-30
daily_headlines  []
date  2013-05-01
daily_headlines  []
date  2013-05-02
daily_headlines  []
date  2013-05-03
daily_headlines  []
date  2013-05-06
daily_headlines  []
date  2013-05-07
daily_headlines  []
date  2013-05-08
daily_headlines  []
date  2013-05-09
daily_headlines  []
date  2013-05-10
daily_headlines  []
date  2013-05-13
daily_headlines  []
date  2013-05-14
daily_headlines  []
date  2013-05-15
daily_headlines  []
date  2013-05-16
daily_headlines  []
date  2013-05-17
daily_headlines  []
date  2013-05-20
daily_headlines  []
date  2013-05-21
daily_headlines  []
date  2013-05-22
daily_headlines  []
date  2013-05-23
daily_headlines  []
date  2013-05-24
daily_headlines  []
date  2013-05-28
daily_headlines  []
date  2013-05-29
d

...

date  2016-01-27
daily_headlines  []
date  2016-01-28
daily_headlines  []
date  2016-01-29
daily_headlines  []
date  2016-02-01
daily_headlines  []
date  2016-02-02
daily_headlines  []
date  2016-02-03
daily_headlines  []
date  2016-02-04
daily_headlines  []
date  2016-02-05
daily_headlines  []
date  2016-02-08
daily_headlines  []
date  2016-02-09
daily_headlines  []
date  2016-02-10
daily_headlines  []
date  2016-02-11
daily_headlines  []
date  2016-02-12
daily_headlines  []
date  2016-02-16
daily_headlines  []
date  2016-02-17
daily_headlines  []
date  2016-02-18
daily_headlines  []
date  2016-02-19
daily_headlines  []
date  2016-02-22
daily_headlines  []
date  2016-02-23
daily_headlines  []
date  2016-02-24
daily_headlines  []
date  2016-02-25
daily_headlines  []
date  2016-02-26
daily_headlines  []
date  2016-02-29
daily_headlines  []
date  2016-03-01
daily_headlines  []
date  2016-03-02
daily_headlines  []
date  2016-03-03
daily_headlines  []
date  2016-03-04
daily_headlines  []
d

daily_headlines  []
date  2017-02-02
daily_headlines  []
date  2017-02-03
daily_headlines  []
date  2017-02-06
daily_headlines  []
date  2017-02-07
daily_headlines  []
date  2017-02-08
daily_headlines  []
date  2017-02-09
daily_headlines  []
date  2017-02-10
daily_headlines  []
date  2017-02-13
daily_headlines  []
date  2017-02-14
daily_headlines  []
date  2017-02-15
daily_headlines  []
date  2017-02-16
daily_headlines  []
date  2017-02-17
daily_headlines  []
date  2017-02-21
daily_headlines  []
date  2017-02-22
daily_headlines  []
date  2017-02-23
daily_headlines  []
date  2017-02-24
daily_headlines  []
date  2017-02-27
daily_headlines  []
date  2017-02-28
daily_headlines  []
date  2017-03-01
daily_headlines  []
date  2017-03-02
daily_headlines  []
date  2017-03-03
daily_headlines  []
date  2017-03-06
daily_headlines  []
date  2017-03-07
daily_headlines  []
date  2017-03-08
daily_headlines  []
date  2017-03-09
daily_headlines  []
date  2017-03-10
daily_headlines  []
date  2017-03-13
d

date  2018-01-10
daily_headlines  []
date  2018-01-11
daily_headlines  []
date  2018-01-12
daily_headlines  []
date  2018-01-16
daily_headlines  []
date  2018-01-17
daily_headlines  []
date  2018-01-18
daily_headlines  []
date  2018-01-19
daily_headlines  []
date  2018-01-22
daily_headlines  []
date  2018-01-23
daily_headlines  []
date  2018-01-24
daily_headlines  []
date  2018-01-25
daily_headlines  []
date  2018-01-26
daily_headlines  []
date  2018-01-29
daily_headlines  []
date  2018-01-30
daily_headlines  []
date  2018-01-31
daily_headlines  []
date  2018-02-01
daily_headlines  []
date  2018-02-02
daily_headlines  []
date  2018-02-05
daily_headlines  []
date  2018-02-06
daily_headlines  []
date  2018-02-07
daily_headlines  []
date  2018-02-08
daily_headlines  []
date  2018-02-09
daily_headlines  []
date  2018-02-12
daily_headlines  []
date  2018-02-13
daily_headlines  []
date  2018-02-14
daily_headlines  []
date  2018-02-15
daily_headlines  []
date  2018-02-16
daily_headlines  []
d

date  2018-12-21
daily_headlines  []
date  2018-12-24
daily_headlines  []
date  2018-12-26
daily_headlines  []
date  2018-12-27
daily_headlines  []
date  2018-12-28
daily_headlines  []
date  2018-12-31
daily_headlines  []
date  2019-01-02
daily_headlines  []
date  2019-01-03
daily_headlines  []
date  2019-01-04
daily_headlines  []
date  2019-01-07
daily_headlines  []
date  2019-01-08
daily_headlines  []
date  2019-01-09
daily_headlines  []
date  2019-01-10
daily_headlines  []
date  2019-01-11
daily_headlines  []
date  2019-01-14
daily_headlines  []
date  2019-01-15
daily_headlines  []
date  2019-01-16
daily_headlines  []
date  2019-01-17
daily_headlines  []
date  2019-01-18
daily_headlines  []
date  2019-01-22
daily_headlines  []
date  2019-01-23
daily_headlines  []
date  2019-01-24
daily_headlines  []
date  2019-01-25
daily_headlines  []
date  2019-01-28
daily_headlines  []
date  2019-01-29
daily_headlines  []
date  2019-01-30
daily_headlines  []
date  2019-01-31
daily_headlines  []
d

date  2019-12-30
daily_headlines  []
date  2019-12-31
daily_headlines  []
date  2020-01-02
daily_headlines  []
date  2020-01-03
daily_headlines  []
date  2020-01-06
daily_headlines  []
date  2020-01-07
daily_headlines  []
date  2020-01-08
daily_headlines  []
date  2020-01-09
daily_headlines  []
date  2020-01-10
daily_headlines  []
date  2020-01-13
daily_headlines  []
date  2020-01-14
daily_headlines  []
date  2020-01-15
daily_headlines  []
date  2020-01-16
daily_headlines  []
date  2020-01-17
daily_headlines  []
date  2020-01-21
daily_headlines  []
date  2020-01-22
daily_headlines  []
date  2020-01-23
daily_headlines  []
date  2020-01-24
daily_headlines  []
date  2020-01-27
daily_headlines  []
date  2020-01-28
daily_headlines  []
date  2020-01-29
daily_headlines  []
date  2020-01-30
daily_headlines  []
date  2020-01-31
daily_headlines  []
date  2020-02-03
daily_headlines  []
date  2020-02-04
daily_headlines  []
date  2020-02-05
daily_headlines  []
date  2020-02-06
daily_headlines  []
d

In [53]:
headlines

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],


<table size="100">
    <tr>
        <td>headlines</td>
        <td>price</td>
    </tr>
    <tr>
        <td>headline-1, headline-2 ..., headline-n</td>
        <td>211.48</td>
    </tr>
</table>

In [23]:
# Check how headlines look like
headlines[:1], price[:1]

([[]], [-297.58007799999905])

## Clean up the price list

In [24]:
price[:2]

[-297.58007799999905, -52.03027300000031]

In [25]:
# Normalize opening prices (target values)
max_price = max(price)
min_price = min(price)
mean_price = np.mean(price)
def normalize(price):
    return ((price-min_price)/(max_price-min_price))

In [26]:
norm_price = []
for p in price:
    norm_price.append(normalize(p))

In [27]:
# Check that normalization worked well
print(min(norm_price))
print(max(norm_price))
print(np.mean(norm_price))

0.0
1.0
0.4311136415569895


In [28]:
# Compare the number of headlines for each day
print(max(len(i) for i in headlines))
print(min(len(i) for i in headlines))
print(np.mean([len(i) for i in headlines]))

0
0
0.0


In [29]:
norm_price[:2]

[0.3626470378121623, 0.4202374317915498]

## Clean up the headlines list

In [30]:
# remove contractions
def decontracted(phrase):
    if "'" in phrase:
        # specific
        phrase = re.sub(r"won't", "will not", phrase)
        phrase = re.sub(r"can\'t", "can not", phrase)

        # general
        phrase = re.sub(r"n\'t", " not", phrase)
        phrase = re.sub(r"\'re", " are", phrase)
        phrase = re.sub(r"\'s", " is", phrase)
        phrase = re.sub(r"\'d", " would", phrase)
        phrase = re.sub(r"\'ll", " will", phrase)
        phrase = re.sub(r"\'t", " not", phrase)
        phrase = re.sub(r"\'ve", " have", phrase)
        phrase = re.sub(r"\'m", " am", phrase)
    return phrase

text = "I should've gone to dentist so my teeth wouldn't hurt"
text1 = "But I am good now"
print(decontracted(text))
print(decontracted(text1))

I should have gone to dentist so my teeth would not hurt
But I am good now


In [47]:
def clean_text(text):
    '''Remove unwanted characters and format the text to create fewer nulls word embeddings'''
    
    # Convert words to lower case
    text = text.lower()
    
    # Replace contractions with their longer forms 
    if True:
        text = text.split()
        new_text = []
        # Remove the contractions
        for word in text:
            new_text.append(decontracted(word))
        # Recreate the sentence
        text = " ".join(new_text)
    
    # Format words and remove unwanted characters
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'0,0', '00', text) 
    text = re.sub(r'[_"\-;%()|.,+&=*%.,!?:#@\[\]]', ' ', text)
    text = re.sub(r'\'', ' ', text)
    text = re.sub(r'\$', ' $ ', text)
    text = re.sub(r'u s ', ' united states ', text)
    text = re.sub(r'u n ', ' united nations ', text)
    text = re.sub(r'u k ', ' united kingdom ', text)
    text = re.sub(r'j k ', ' jk ', text)
    text = re.sub(r' s ', ' ', text)
    text = re.sub(r' yr ', ' year ', text)
    text = re.sub(r' l g b t ', ' lgbt ', text)
    text = re.sub(r'0km ', '0 km ', text)
    
    # Remove stop words
    text = text.split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)

    return text

In [48]:
# Clean the headlines
clean_headlines = []

for daily_headlines in headlines:
    clean_daily_headlines = []
    for headline in daily_headlines:
        clean_daily_headlines.append(clean_text(headline))
    clean_headlines.append(clean_daily_headlines)

In [49]:
clean_headlines[0]

[]

In [50]:
# Take a look at some headlines to ensure everything was cleaned well
clean_headlines[:2]

[[], []]

In [51]:
print('Roughly the number of unique words in English: {}'.format(len({word: None 
                                                                      for headlines in clean_headlines 
                                                                      for headline in headlines 
                                                                      for word in headline.split()})))


Roughly the number of unique words in English: 0


In [36]:
# Create the word vocab
import collections
words = [word for headlines in clean_headlines for headline in headlines for word in headline.split()]
word_counts = collections.Counter(words)

In [37]:
word_counts

Counter()

## using Glove embeddings to initialize our weights while designing our neural network.

In [39]:
# Load GloVe's embeddings
embeddings_index = {}
with open('glove.840B.300d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        embedding = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = embedding

print('Word embeddings:', len(embeddings_index))

Word embeddings: 2196016


In [40]:
# Limit the vocab that we will use to words that appear ≥ threshold or are in GloVe

# Define threshold
threshold = 10

#dictionary to convert words to integers
vocab_to_int = {} 

value = 0
for word, count in word_counts.items():
    if count >= threshold or word in embeddings_index:
        vocab_to_int[word] = value
        value += 1

In [41]:
len(vocab_to_int)

0

In [43]:
# Special tokens that will be added to our vocab
codes = ["<UNK>","<PAD>"]   

# Add codes to vocab
for code in codes:
    vocab_to_int[code] = len(vocab_to_int)

# Dictionary to convert integers to words
int_to_vocab = {}
for word, value in vocab_to_int.items():
    int_to_vocab[value] = word

usage_ratio = round(len(vocab_to_int) / len(word_counts),4)*100

print("Total Number of Unique Words:", len(word_counts))
print("Number of Words we will use:", len(vocab_to_int))
print("Percent of Words we will use: {}%".format(usage_ratio))

ZeroDivisionError: division by zero

## For the words which are common within headlines but are absent in Glove corpus,  are randomly initialized .

In [44]:
# Need to use 300 for embedding dimensions to match GloVe's vectors.
embedding_dim = 300

nb_words = len(vocab_to_int)
# Create matrix with default values of zero
word_embedding_matrix = np.zeros((nb_words, embedding_dim))
for word, i in vocab_to_int.items():
    if word in embeddings_index:
        word_embedding_matrix[i] = embeddings_index[word]
    else:
        # If word not in GloVe, create a random embedding for it
        new_embedding = np.array(np.random.uniform(-1.0, 1.0, embedding_dim))
        embeddings_index[word] = new_embedding
        word_embedding_matrix[i] = new_embedding

# Check if value matches len(vocab_to_int)
print(len(word_embedding_matrix))

IndexError: index 2 is out of bounds for axis 0 with size 2

##  word sequences are converted to equivalent integer sequences so that it can be used as input to the model

In [45]:
# Change the text from words to integers
# If word is not in vocab, replace it with <UNK> (unknown)
word_count = 0
unk_count = 0

headlines_sequence = []

for daily_headline in clean_headlines:
    daily_headlines_seq = []
    for headline in daily_headline:
        headline_seq = []
        for word in headline.split():
            word_count += 1
            if word in vocab_to_int:
                headline_seq.append(vocab_to_int[word])
            else:
                headline_seq.append(vocab_to_int["<UNK>"])
                unk_count += 1
        daily_headlines_seq.append(headline_seq)
    headlines_sequence.append(daily_headlines_seq)

unk_percent = round(unk_count/word_count,4)*100

print("Total number of words in headlines:", word_count)
print("Total number of UNKs in headlines:", unk_count)
print("Percent of words that are UNK: {}%".format(unk_percent))

ZeroDivisionError: division by zero

In [46]:
headlines_sequence[:1]

[[]]

In [44]:
# Find the length of headlines
lengths = []
for headlines in headlines_sequence:
    for headline in headlines:
        lengths.append(len(headline))

# Create a dataframe so that the values can be inspected
lengths = pd.DataFrame(lengths, columns=['counts'])

In [45]:
lengths

,counts
0,20
1,19
2,7
3,19
4,10
...,...
49688,11
49689,7
49690,7
49691,7


In [46]:
lengths.describe()

,counts
count,49693.000000
mean,12.409917
std,6.789827
min,1.000000
25%,7.000000
50%,10.000000
75%,16.000000
max,41.000000


In [47]:
max_headline_length = 16
max_daily_length = 200
pad_headlines = []

# For each date in all the dates available
for headlines in headlines_sequence:
    pad_daily_headlines = []
    # for each headline for each date
    for headline in headlines:
        # Add headline if it is less than max length
        if len(headline) <= max_headline_length:
            for word in headline:
                pad_daily_headlines.append(word)
        # Limit headline if it is more than max length  
        else:
            headline = headline[:max_headline_length]
            for word in headline:
                pad_daily_headlines.append(word)
    
    # Pad daily_headlines if they are less than max length
    if len(pad_daily_headlines) < max_daily_length:
        for i in range(max_daily_length-len(pad_daily_headlines)):
            pad = vocab_to_int["<PAD>"]
            pad_daily_headlines.append(pad)
    # Limit daily_headlines if they are more than max length
    else:
        pad_daily_headlines = pad_daily_headlines[:max_daily_length]
    pad_headlines.append(pad_daily_headlines)

In [48]:
pad_headlines[:1]

[[0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  20,
  21,
  22,
  23,
  31295,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  30,
  32,
  33,
  37,
  38,
  22,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  22,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  102,
  72,
  143,
  144,
  145,
  146,
  150,
  151,
  152,
  153,
  154,
  155,
  156,
  157,
  158,
  159,
  160,
  161,
  162,
  16

## Split data into training and testing sets.
## Validating data will be created during training.

In [49]:
x_train, x_test, y_train, y_test = train_test_split(pad_headlines, norm_price, test_size = 0.15, random_state = 2)

x_train = np.array(x_train)
x_test = np.array(x_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [50]:
# Check the lengths
print(len(x_train))
print(len(x_test))

1689
299


# Model Building

## 1. Define the hyperparameters

In [51]:
filter_length = 5
dropout = 0.5
learning_rate = 0.001
weights = initializers.TruncatedNormal(mean=0.0, stddev=0.1, seed=2)
nb_filter = 16
rnn_output_size = 128
hidden_dims = 128

## 2. Create the model

In [52]:
def build_model():
    
    model = Sequential()
    
    # Layer 1 - Embedding
    model.add(Embedding(nb_words, 
                         embedding_dim,
                         weights=[word_embedding_matrix], 
                         input_length=max_daily_length))
    model.add(Dropout(dropout))
    
    # Layer 2 - Convolution 1 with dropout
    model.add(Convolution1D(filters = nb_filter, 
                             kernel_size = filter_length, 
                             padding = 'same',
                             activation = 'relu'))
    model.add(Dropout(dropout))    

    # Layer 3 - Convolution 2 with Dropout 
    model.add(Convolution1D(filters = nb_filter, 
                                 kernel_size = filter_length, 
                                 padding = 'same',
                                 activation = 'relu'))
    model.add(Dropout(dropout))    

    # Layer 4 - RNN with dropout
    model.add(LSTM(rnn_output_size, 
                    activation=None,
                    kernel_initializer=weights,
                    dropout = dropout))    

    # Layer 5 - Dense FFN with Dropout
    model.add(Dense(hidden_dims, kernel_initializer=weights))
    model.add(Dropout(dropout))
    
    model.add(Dense(1, 
                    kernel_initializer = weights,
                    name='output'))

    model.compile(loss='mean_squared_error',
                  optimizer=Adam(lr=learning_rate,clipvalue=1.0))
    return model

## 3. Fit the model

In [53]:
model = build_model()
print()
save_best_weights = 'best_weights.h5'

callbacks = [ModelCheckpoint(save_best_weights, monitor='val_loss', save_best_only=True),
            EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto'),
            ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=3)]

history = model.fit([x_train],
                    y_train,
                    batch_size=128,
                    epochs=100,
                    validation_split=0.15,
                    verbose=True,
                    shuffle=True,
                    callbacks = callbacks)
print(model.summary())






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 1435 samples, validate on 254 samples
Epoch 1/100
1435/1435 [==============================] - 8s 6ms/step - loss: 0.0868 - val_loss: 0.0801
Epoch 2/100
1435/1435 [==============================] - 4s 3ms/step - loss: 0.0338 - val_loss: 0.0368
Epoch 3/100
1435/1435 [==============================] - 4s 3ms/step - loss: 0.0207 - val_loss: 0.0233
Epoch 4/100
1435/1435 [==============================] - 4s 3ms/step - loss: 0.0162 - val_loss: 0.0166
Epoch 5/100
1435/1435 [==============================] - 4s 3ms/step - loss: 0.0134 - val_loss: 0.0109
Epoch 6/100
1435/1435 [==============================] - 4s 3ms/step - loss: 0.0117 - val_loss: 0.0082
Epoch 7/100
1435/1435 [==============================] - 4s 3ms/step - loss: 0.0110 - val_loss: 0.0092
Epoch 8/100
1435

## 4. Predict using the model

In [54]:
predictions = model.predict([x_test], verbose = True)

299/299 [==============================] - 2s 6ms/step


In [55]:
predictions

array([[0.44225883],
       [0.4424222 ],
       [0.4386402 ],
       [0.4424549 ],
       [0.4396373 ],
       [0.43782273],
       [0.43942687],
       [0.44058594],
       [0.43952933],
       [0.4425978 ],
       [0.43791977],
       [0.43852067],
       [0.4377684 ],
       [0.43848366],
       [0.4378564 ],
       [0.43796563],
       [0.4377485 ],
       [0.43879303],
       [0.4417506 ],
       [0.44086352],
       [0.44175854],
       [0.4387307 ],
       [0.43980727],
       [0.43829173],
       [0.43947804],
       [0.44126692],
       [0.43961182],
       [0.44152176],
       [0.44331878],
       [0.4383184 ],
       [0.44166616],
       [0.44285363],
       [0.43737414],
       [0.4405006 ],
       [0.43867728],
       [0.441166  ],
       [0.4385178 ],
       [0.43743688],
       [0.43966186],
       [0.43833932],
       [0.4412713 ],
       [0.43876773],
       [0.44060627],
       [0.44091466],
       [0.44351926],
       [0.4392978 ],
       [0.44229975],
       [0.440

In [56]:
y_test

array([0.43612946, 0.60945737, 0.41281364, 0.46068313, 0.40736662,
       0.41510955, 0.45121826, 0.43220497, 0.46469636, 0.44856275,
       0.52025575, 0.37891786, 0.46430737, 0.45260537, 0.49764819,
       0.45788643, 0.47788516, 0.52623353, 0.37129364, 0.53228219,
       0.54641552, 0.42023155, 0.47802116, 0.44470339, 0.47763736,
       0.46047105, 0.52224393, 0.43062336, 0.51963567, 0.45707214,
       0.46076669, 0.4051658 , 0.46460242, 0.34425446, 0.52781485,
       0.48596406, 0.45300589, 0.47002524, 0.46413045, 0.3803868 ,
       0.41834941, 0.42859917, 0.43965227, 0.45238062, 0.05600198,
       0.51086696, 0.43250637, 0.52945553, 0.45700183, 0.38942176,
       0.42567223, 0.46518043, 0.49425562, 0.4264335 , 0.44337621,
       0.36968812, 0.46944723, 0.55163203, 0.45382075, 0.10422588,
       0.39579487, 0.45755622, 0.49511024, 0.44707537, 0.46691504,
       0.5366314 , 0.51926455, 0.45871282, 0.41833789, 0.3797673 ,
       0.51143978, 0.53556931, 0.23660028, 0.49309902, 0.49979

In [57]:
# Compare testing loss to training and validating loss
mse(y_test, predictions)

0.0070940205570736

In [58]:
# # Revert prediction back to actual scale
# def unnormalize(price):
#     '''Revert values to their unnormalized amounts'''
#     price = price*(max_price-min_price)+min_price
#     return(price)

In [59]:
# # Store back-scaled predictions
# unnorm_predictions = []
# for pred in predictions:
#     unnorm_predictions.append(unnormalize(pred))

# # Store back-scaled actuals
# unnorm_y_test = []
# for y in y_test:
#     unnorm_y_test.append(unnormalize(y))

In [60]:
# Calculate the median absolute error for the predictions
# mae(unnorm_y_test, unnorm_predictions)

In [61]:
#pd.Series(unnorm_y_test).describe()

In [62]:
# def news_to_int(news):
#     '''Convert your created news into integers'''
#     ints = []
#     for word in news.split():
#         if word in vocab_to_int:
#             ints.append(vocab_to_int[word])
#         else:
#             ints.append(vocab_to_int['<UNK>'])
#     return ints

In [63]:
# def padding_news(news):
#     '''Adjusts the length of your created news to fit the model's input values.'''
#     padded_news = news
#     if len(padded_news) < max_daily_length:
#         for i in range(max_daily_length-len(padded_news)):
#             padded_news.append(vocab_to_int["<PAD>"])
#     elif len(padded_news) > max_daily_length:
#         padded_news = padded_news[:max_daily_length]
#     return padded_news

In [64]:
# # Default news that you can use

# create_news =  "Woman says note from Chinese 'prisoner' was hidden in new purse. \
#                21,000 AT&T workers poised for Monday strike \
#                housands march against Trump climate policies in D.C., across USA \
#                Kentucky judge won't hear gay adoptions because it's not in the child's \"best interest\" \
#                Multiple victims shot in UTC area apartment complex \
#                Drones Lead Police to Illegal Dumping in Riverside County | NBC Southern California \
#                An 86-year-old Californian woman has died trying to fight a man who was allegedly sexually assaulting her 61-year-old friend. \
#                Fyre Festival Named in $5Million+ Lawsuit after Stranding Festival-Goers on Island with Little Food, No Security. \
#                The \"Greatest Show on Earth\" folds its tent for good \
#                U.S.-led fight on ISIS have killed 352 civilians: Pentagon \
#                Woman offers undercover officer sex for $25 and some Chicken McNuggets \
#                Ohio bridge refuses to fall down after three implosion attempts \
#                Jersey Shore MIT grad dies in prank falling from library dome \
#                New York graffiti artists claim McDonald's stole work for latest burger campaign \
#                SpaceX to launch secretive satellite for U.S. intelligence agency \
#                Severe Storms Leave a Trail of Death and Destruction Through the U.S. \
#                Hamas thanks N. Korea for its support against ‘Israeli occupation’ \
#                Baker Police officer arrested for allegedly covering up details in shots fired investigation \
#                Miami doctor’s call to broker during baby’s delivery leads to $33.8 million judgment \
#                Minnesota man gets 15 years for shooting 5 Black Lives Matter protesters \
#                South Australian woman facing possible 25 years in Colombian prison for drug trafficking \
#                The Latest: Deal reached on funding government through Sept. \
#                Russia flaunts Arctic expansion with new military bases"

# clean_news = clean_text(create_news)

# int_news = news_to_int(clean_news)

# pad_news = padding_news(int_news)

# pad_news = np.array(pad_news).reshape((1,-1))

# pred = model.predict([pad_news])

# price_change = unnormalize(pred)

# print("The Dow should open: {} from the previous open.".format(np.round(price_change[0][0],2)))